In [1]:
from collections import namedtuple
from Crypto.Util.number import inverse, bytes_to_long

In [2]:
Point = namedtuple("Point", "x y")
O = 'INFINITY'

p = 9631668579539701602760432524602953084395033948174466686285759025897298205383
gx = 5664314881801362353989790109530444623032842167510027140490832957430741393367
gy = 3735011281298930501441332016708219762942193860515094934964869027614672869355
G = Point(gx, gy)
A = Point(x=3829488417236560785272607696709023677752676859512573328792921651640651429215, y=7947434117984861166834877190207950006170738405923358235762824894524937052000)
B = Point(x=9587224500151531060103223864145463144550060225196219072827570145340119297428, y=2527809441042103520997737454058469252175392602635610992457770946515371529908)

In [3]:
F.<a, b> = PolynomialRing(Zmod(p))
I = ideal([G.x ** 3 + a * G.x + b - G.y ** 2, A.x ** 3 + a * A.x + b - A.y ** 2])
I.variety()

verbose 0 (3827: multi_polynomial_ideal.py, groebner_basis) Warning: falling back to very slow toy implementation.
verbose 0 (1081: multi_polynomial_ideal.py, dimension) Warning: falling back to very slow toy implementation.
verbose 0 (2264: multi_polynomial_ideal.py, variety) Warning: falling back to very slow toy implementation.


[{a: 9605275265879631008726467740646537125692167794341640822702313056611938432994, b: 7839838607707494463758049830515369383778931948114955676985180993569200375480}]

In [14]:
a = 9605275265879631008726467740646537125692167794341640822702313056611938432994
b = 7839838607707494463758049830515369383778931948114955676985180993569200375480
(4*a^3 + 27*b^2)%p

0

In [15]:
def is_on_curve(P):
    if P == O:
        return True
    else:
        return (P.y**2 - (P.x**3 + a*P.x + b)) % p == 0 and 0 <= P.x < p and 0 <= P.y < p

is_on_curve(G), is_on_curve(A), is_on_curve(B)

(True, True, True)

In [16]:
F.<x> = PolynomialRing(Zmod(p))
r = (x^3+a*x+b).roots()
r

[(7925182757193285961316626419940151258451119718064102936455321651294650340555,
  1),
 (853242911173207820721903052331400912971957115055181874915218687301323932414,
  2)]

In [17]:
alpha = Zmod(p)(853242911173207820721903052331400912971957115055181874915218687301323932414)
beta = Zmod(p)(7925182757193285961316626419940151258451119718064102936455321651294650340555)
(alpha ^ 3 + a*alpha + b), (beta ^ 3 + a*beta + b)

(0, 0)

In [18]:
def point_inverse(P):
    if P == O:
        return P
    return Point(P.x, -P.y % p)

def point_addition(P, Q):
    if P == O:
        return Q
    elif Q == O:
        return P
    elif Q == point_inverse(P):
        return O
    else:
        if P == Q:
            s = (3*P.x**2 + a)*inverse(2*P.y, p) % p
        else:
            s = (Q.y - P.y) * inverse((Q.x - P.x), p) % p
    Rx = (s**2 - P.x - Q.x) % p
    Ry = (s*(P.x - Rx) - P.y) % p
    R = Point(Rx, Ry)
    assert is_on_curve(R)
    return R

def point_multiply(P, d):
    bits = bin(d)[2:]
    Q = O
    for bit in bits:
        Q = point_addition(Q, Q)
        if bit == '1':
            Q = point_addition(Q, P)
    assert is_on_curve(Q)
    return Q

In [19]:
def phi(P):
    t = (alpha - beta).sqrt() * (P.x - alpha)
    return (P.y + t) / (P.y - t)

print(phi(point_addition(A,B)) == phi(A) * phi(B))

True


In [20]:
dA = discrete_log(phi(A), phi(G))
dA

1532487521612462894579517163606359285989568203515734083099567402780433190052

In [21]:
k = point_multiply(B, dA).x
k

195244836939422730384635168512423975722005537834468919742841431561900513012

In [22]:
import hashlib

k = hashlib.sha512(str(k).encode('ascii')).digest()
flag = bytes.fromhex('1536c5b019bd24ddf9fc50de28828f727190ff121b709a6c63c4f823ec31780ad30d219f07a8c419c7afcdce900b6e89b37b18b6daede22e5445eb98f3ca2e40')
bytes(ci ^^ ki for ci, ki in zip(flag, k))

b'FLAG{adbffefdb46a99fad0042dd3c10fdc414fadd25c}\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'